In [5]:
import importlib
import sys
import os
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["TORCH_NUM_THREADS"] = "1"
import torch
import pickle

sys.path.insert(0, '..')
sys.path.insert(0, '../..')
sys.path.insert(0, '../../..')
sys.path.insert(0, '../../../..')

from model.dropout_uncertainty_enc_dec_LSTM.dropout_uncertainty_model import DropoutUncertaintyEncoderDecoderLSTM

In [6]:
# Load model
file_path_model = '../../../training_variational_dropout/Sepsis/Sepsis_full_grad_norm_proactive_conf_check_02.pkl'
model = DropoutUncertaintyEncoderDecoderLSTM.load(file_path_model, dropout=0.1)

# Load the dataset
file_path_data_set = '../../../../../../risk_controlled_proactive_conformance_checking_dev/data/encoded_data/Sepsis/Sepsis_all_normal_3_val.pkl'
sepsis_test_dataset = torch.load(file_path_data_set, weights_only=False)

Data set categories:  ([('concept:name', 18, {'Admission IC': 1, 'Admission NC': 2, 'CRP': 3, 'EOS': 4, 'ER Registration': 5, 'ER Sepsis Triage': 6, 'ER Triage': 7, 'IV Antibiotics': 8, 'IV Liquid': 9, 'LacticAcid': 10, 'Leucocytes': 11, 'Release A': 12, 'Release B': 13, 'Release C': 14, 'Release D': 15, 'Release E': 16, 'Return ER': 17}), ('InfectionSuspected', 5, {'EOS': 1, 'False': 2, 'True': 3, nan: 4}), ('org:group', 27, {'?': 1, 'A': 2, 'B': 3, 'C': 4, 'D': 5, 'E': 6, 'EOS': 7, 'F': 8, 'G': 9, 'H': 10, 'I': 11, 'J': 12, 'K': 13, 'L': 14, 'M': 15, 'N': 16, 'O': 17, 'P': 18, 'Q': 19, 'R': 20, 'S': 21, 'T': 22, 'U': 23, 'V': 24, 'W': 25, 'X': 26}), ('DiagnosticBlood', 5, {'EOS': 1, 'False': 2, 'True': 3, nan: 4}), ('DisfuncOrg', 5, {'EOS': 1, 'False': 2, 'True': 3, nan: 4}), ('SIRSCritTachypnea', 5, {'EOS': 1, 'False': 2, 'True': 3, nan: 4}), ('Hypotensie', 5, {'EOS': 1, 'False': 2, 'True': 3, nan: 4}), ('SIRSCritHeartRate', 5, {'EOS': 1, 'False': 2, 'True': 3, nan: 4}), ('Infusion'

In [7]:
import evaluation.probabilistic_evaluation
importlib.reload(evaluation.probabilistic_evaluation)
from evaluation.probabilistic_evaluation import ProbabilisticEvaluation

new_eval = ProbabilisticEvaluation(model, 
                                   sepsis_test_dataset,
                                   # Change here the number of cores for workers:
                                   num_processes=32,
                                   # growing_num_values = [],
                                   growing_num_values = ['case_elapsed_time'],
                                   # Change here number of samples:
                                   samples_per_case = 100,
                                   sample_argmax = False,
                                   use_variance_cat = True,
                                   use_variance_num = True,
                                   all_cat=['concept:name'],
                                   # all_num=['case_elapsed_time', 'event_elapsed_time'])
                                   all_num=['case_elapsed_time'])

In [8]:
def save_chunk(results, i):
    chunk_number = (i + 1)
    filename = os.path.join(output_dir, f'results_part_{chunk_number:03d}.pkl')
    with open(filename, 'wb') as f:
        pickle.dump(results, f)
    print(f"Saved {len(results)} results to {filename}")

output_dir = '../../../../../../../data/Sepsis/proact_conf_check/eval_validation_set/'

save_every = 50
results = {}
for i, (case_name, prefix_len, prefix, predicted_suffixes, suffix, mean_prediction) in enumerate(new_eval.evaluate_multi_processing(random_order=True, include_model_states=False)):
    # print(case_name, prefix_len)
    assert((case_name, prefix_len) not in results)
    results[(case_name, prefix_len)] = (prefix, suffix, mean_prediction, predicted_suffixes)
    # print(prefix_len, len(suffix))
    if (i + 1) % save_every == 0:
        save_chunk(results, i)
        results = {}

if len(results):
    save_chunk(results, i)

  0%|          | 0/314 [00:00<?, ?it/s]

Saved 50 results to ../../../../../../../data/Sepsis/proact_conf_check/eval_validation_set/results_part_050.pkl
Saved 50 results to ../../../../../../../data/Sepsis/proact_conf_check/eval_validation_set/results_part_100.pkl
Saved 50 results to ../../../../../../../data/Sepsis/proact_conf_check/eval_validation_set/results_part_150.pkl
Saved 50 results to ../../../../../../../data/Sepsis/proact_conf_check/eval_validation_set/results_part_200.pkl
Saved 50 results to ../../../../../../../data/Sepsis/proact_conf_check/eval_validation_set/results_part_250.pkl
Saved 50 results to ../../../../../../../data/Sepsis/proact_conf_check/eval_validation_set/results_part_300.pkl
Saved 50 results to ../../../../../../../data/Sepsis/proact_conf_check/eval_validation_set/results_part_350.pkl
Saved 50 results to ../../../../../../../data/Sepsis/proact_conf_check/eval_validation_set/results_part_400.pkl
Saved 50 results to ../../../../../../../data/Sepsis/proact_conf_check/eval_validation_set/results_part_